# Go to Yahoo! Finances URL and scrape information from the "Statistics" tab

This Python script fetches financial statistics data from a Yahoo Finance page for a specific company, with the ticker symbol "MSFT" (Microsoft) in this example. However, any publicly traded company's ticker can be used. It uses the requests library to send an HTTP GET request to the specified URL, including a user-agent header to mimic a browser request. Upon receiving a successful response (status code 200), it uses BeautifulSoup to parse the HTML content and extract all tables on the page. For each table, it iterates through the rows and columns to retrieve the metric names and their corresponding values, storing these in a dictionary. These dictionaries are then converted into pandas DataFrames. All DataFrames are concatenated into a single DataFrame, result_df, which holds the combined financial statistics.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np

In [ ]:


# URL of the Yahoo Finance statistics page for a specific company
# Replace the ticker with whatever company you want to get data for
url = "https://finance.yahoo.com/quote/MSFT/key-statistics"

# Rate limit in seconds to respect the server's resources
RATE_LIMIT = 5

def get_statistics(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all the tables containing the data
        tables = soup.find_all('table')
        data_frames = []

        for table in tables:
            # Find all rows in the table
            rows = table.find_all('tr')
            data = {'Metric': [], 'Value': []}

            for row in rows:
                # Find all columns in each row
                cols = row.find_all('td')
                if len(cols) > 1:
                    metric = cols[0].text.strip()
                    value = cols[1].text.strip()
                    data['Metric'].append(metric)
                    data['Value'].append(value)

            # Create a DataFrame for the current table
            df = pd.DataFrame(data)
            data_frames.append(df)

        # Combine all DataFrames into one
        result_df = pd.concat(data_frames, ignore_index=True)
        return result_df
    else:
        print(f"Error: Unable to fetch data. Status code: {response.status_code}")
        return pd.DataFrame()

#Final data is stored in df_statistics

if __name__ == "__main__":
    df_statistics = get_statistics(url)
    print(df_statistics)
    time.sleep(RATE_LIMIT)  # Respect the rate limit

Save the resulting dataframe to a csv file:

In [ ]:
# output df_statistics to a csv

df_statistics.to_csv('statistics.csv')


# Get data from the financials tab in Yahoo! Finance

This next chunk of code is designed to fetch financial data for a publicly traded company that exists on Yahoo!. It begins by defining the URL. The script then extracts the ticker symbol from the URL by locating the substring between "/quote/" and the next "/". A function named get_financials is defined, which sends an HTTP GET request to the provided URL with a user-agent header to mimic a browser. If the request is successful (status code 200), the HTML content is parsed using BeautifulSoup. The script searches for table rows within div elements that match a specific class (assumed to be "D(tbr)"). It treats the first row as containing column headers and the remaining rows as data rows, extracting text from each div within the rows. These are compiled into lists, and a pandas DataFrame is created with the first column set as the index ("Breakdown"). If the request fails, an empty DataFrame is returned. 

In [ ]:


url = "https://finance.yahoo.com/quote/GILD/financials"

# Extract ticker from URL
# The ticker is between "/quote/" and the next "/"
start = url.find("/quote/") + len("/quote/")
end = url.find("/", start)
ticker = url[start:end]


def get_financials(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the table. This requires correct selectors for the actual table.
        # Assuming the table rows are in 'div' with a specific class (update class as per actual HTML structure)
        table_rows = soup.find_all('div', class_='D(tbr)')

        # Assuming the first 'div' of each row contains the row header (label)
        # and the subsequent 'divs' contain the data.
        row_headers = [row.find('div').get_text() for row in table_rows]

        # Assuming the first row contains column headers
        column_headers = ['Breakdown'] + [header.get_text() for header in table_rows[0].find_all('div', recursive=False)][1:]

        # Extracting data starting from the second row, since the first row contains headers
        data = []
        for row in table_rows[1:]:  # Skip the header row
            cols = row.find_all('div', recursive=False)
            row_data = [col.get_text() for col in cols]
            data.append(row_data)

        # Create DataFrame
        df = pd.DataFrame(data, columns=column_headers)
        df.set_index('Breakdown', inplace=True)

        return df
    else:
        print("Failed to fetch the data")
        return pd.DataFrame()

if __name__ == "__main__":
    df_financials = get_financials(url)
    print(df_financials)


                                                           ttm  12/31/2023  \
Breakdown                                                                    
Total Revenue                                       27,116,000  27,116,000   
Cost of Revenue                                      6,498,000   6,498,000   
Gross Profit                                        20,618,000  20,618,000   
Operating Expense                                   11,808,000  11,808,000   
Operating Income                                     8,810,000   8,810,000   
Net Non Operating Interest Income Expense             -944,000    -944,000   
Other Income Expense                                -1,007,000  -1,007,000   
Pretax Income                                        6,859,000   6,859,000   
Tax Provision                                        1,247,000   1,247,000   
Net Income Common Stockholders                       5,665,000   5,665,000   
Diluted NI Available to Com Stockholders             5,665,000  

# Cleaning the data 

For each column in the DataFrame, this code removes dollar signs and commas, replaces cells containing only a dash with NaN, and attempts to convert the values to floats. The index name 'Breakdown' is removed for simplicity, and the index is reset to make 'Breakdown' a regular column. Additionally, a new column 'Ticker' is added to the DataFrame, containing the ticker symbol of the company. Finally, the updated DataFrame, which now includes NaN values where there were originally only dashes, is printed.

In [ ]:
# Convert all number-like strings to floats, remove commas, and replace only dashes with NaN
for column in df_financials.columns:
    df_financials[column] = (
        df_financials[column]
        .replace('[\$,]', '', regex=True)  # remove dollar signs and commas
        .replace('^\-$', np.nan, regex=True)  # replace cells with only a dash with NaN
        .astype(float, errors='ignore')  # convert to float, ignore if conversion is not possible
    )

# Remove the index name 'Breakdown' if not necessary
df_financials.index.name = None

# Reset the index if you want 'Breakdown' as a regular column
df_financials.reset_index(inplace=True)


#show ticker for company
df_financials['Ticker'] = ticker

# Now, 'df_financials' should have NaN where there were only dashes
print(df_financials)


                                                index          ttm  \
0                                       Total Revenue  27116000.00   
1                                     Cost of Revenue   6498000.00   
2                                        Gross Profit  20618000.00   
3                                   Operating Expense  11808000.00   
4                                    Operating Income   8810000.00   
5           Net Non Operating Interest Income Expense   -944000.00   
6                                Other Income Expense  -1007000.00   
7                                       Pretax Income   6859000.00   
8                                       Tax Provision   1247000.00   
9                      Net Income Common Stockholders   5665000.00   
10           Diluted NI Available to Com Stockholders   5665000.00   
11                                          Basic EPS         4.70   
12                                        Diluted EPS         4.67   
13                  

In [ ]:
#write df_financials to a csv

df_financials.to_csv('financials.csv')
df_financials

,index,ttm,12/31/2023,12/31/2022,12/31/2021,12/31/2020,Ticker
0,Total Revenue,27116000.00,27116000.00,27281000.00,27305000.00,24689000.0,GILD
1,Cost of Revenue,6498000.00,6498000.00,5657000.00,6601000.00,4572000.0,GILD
2,Gross Profit,20618000.00,20618000.00,21624000.00,20704000.00,20117000.0,GILD
3,Operating Expense,11808000.00,11808000.00,10650000.00,9847000.00,10190000.0,GILD
4,Operating Income,8810000.00,8810000.00,10974000.00,10857000.00,9927000.0,GILD
5,Net Non Operating Interest Income Expense,-944000.00,-944000.00,-935000.00,-1001000.00,-984000.0,GILD
6,Other Income Expense,-1007000.00,-1007000.00,-4225000.00,-1578000.00,-7274000.0,GILD
7,Pretax Income,6859000.00,6859000.00,5814000.00,8278000.00,1669000.0,GILD
8,Tax Provision,1247000.00,1247000.00,1248000.00,2077000.00,1580000.0,GILD
9,Net Income Common Stockholders,5665000.00,5665000.00,4592000.00,6225000.00,123000.0,GILD


# Final Consolidated Code


I decided to consolidate the code into one code block for convenience purposes, as this was what my teammates and project supervisor wanted.

In [ ]:

def get_soup(url):
    """Fetch the HTML content from a URL and return a BeautifulSoup object."""
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return BeautifulSoup(response.text, 'html.parser')
    else:
        print(f"Failed to fetch the data. Status code: {response.status_code}")
        return None

def get_statistics(soup):
    """Extract statistics data from the BeautifulSoup object and return a DataFrame."""
    tables = soup.find_all('table')
    data_frames = []
    for table in tables:
        rows = table.find_all('tr')
        data = {'Metric': [], 'Value': []}
        for row in rows:
            cols = row.find_all('td')
            if len(cols) > 1:
                metric = cols[0].text.strip()
                value = cols[1].text.strip()
                data['Metric'].append(metric)
                data['Value'].append(value)
        df = pd.DataFrame(data)
        data_frames.append(df)
    return pd.concat(data_frames, ignore_index=True) if data_frames else pd.DataFrame()

def get_financials(soup):
    """Extract financials data from the BeautifulSoup object and return a DataFrame."""
    table_rows = soup.find_all('div', class_='D(tbr)')
    if not table_rows:
        return pd.DataFrame()
    column_headers = ['Breakdown'] + [header.get_text() for header in table_rows[0].find_all('div', recursive=False)][1:]
    data = [[col.get_text() for col in row.find_all('div', recursive=False)] for row in table_rows[1:]]
    df = pd.DataFrame(data, columns=column_headers)
    df.set_index('Breakdown', inplace=True)
    return df

def clean_and_augment_financials(df_financials, ticker):
    """Clean and augment the financials DataFrame."""
    for column in df_financials.columns:
        df_financials[column] = (
            df_financials[column]
            .replace('[\$,]', '', regex=True)
            .replace('^\-$', np.nan, regex=True)
            .astype(float, errors='ignore')
        )
    df_financials.index.name = None
    df_financials.reset_index(inplace=True)
    df_financials['Ticker'] = ticker
    return df_financials

if __name__ == "__main__":
    # Prompt user for a ticker symbol
    ticker = input("Enter the ticker symbol of the company: ").strip().upper()

    # Construct URLs
    statistics_url = f"https://finance.yahoo.com/quote/{ticker}/key-statistics"
    financials_url = f"https://finance.yahoo.com/quote/{ticker}/financials"

    # Fetch and print statistics
    soup_statistics = get_soup(statistics_url)
    if soup_statistics:
        df_statistics = get_statistics(soup_statistics)
        print("\nStatistics DataFrame:")
        print(df_statistics)

    # Fetch, clean, and print financials
    soup_financials = get_soup(financials_url)
    if soup_financials:
        df_financials = get_financials(soup_financials)
        df_financials = clean_and_augment_financials(df_financials, ticker)
        print("\nFinancials DataFrame:")
        print(df_financials)


Enter the ticker symbol of the company: GILD

Statistics DataFrame:
                                            Metric         Value
0                            Market Cap (intraday)        92.16B
1                                 Enterprise Value       109.89B
2                                     Trailing P/E         16.44
3                                      Forward P/E         10.48
4                        PEG Ratio (5 yr expected)          0.46
5                                Price/Sales (ttm)          3.43
6                                 Price/Book (mrq)          4.04
7                         Enterprise Value/Revenue          4.05
8                          Enterprise Value/EBITDA         10.47
9                                Beta (5Y Monthly)          0.20
10                                52-Week Change 3        -6.07%
11                         S&P500 52-Week Change 3        32.31%
12                                  52 Week High 3         87.87
13                    